In [1]:
%pip install pulp

In [2]:
import pandas as pd
import pulp

In [3]:
# =========================================================
# 1. LOAD DATA
# =========================================================
df_demand = pd.read_csv("sarima_forecast.csv")   # store, product, week, forecast_demand
df_plant_dc = pd.read_csv("plant_dc_cost.csv")   # from_plant, to_dc, product, cost_per_unit
df_dc_store = pd.read_csv("dc_store_cost.csv")   # from_dc, to_store, product, cost_per_unit

In [4]:
# =========================================================
# 2. DEFINE SETS
# =========================================================
plants = df_plant_dc["from_plant"].unique()
dcs = df_plant_dc["to_dc"].unique()
stores = df_demand["store"].unique()
products = df_demand["product"].unique()
weeks = df_demand["week"].unique()

In [5]:
# =========================================================
# 3. COST DICTIONARIES
# =========================================================
cost_p2dc = {
    (r["from_plant"], r["to_dc"], r["product"]): r["cost_per_unit"]
    for _, r in df_plant_dc.iterrows()
}

cost_dc2s = {
    (r["from_dc"], r["to_store"], r["product"]): r["cost_per_unit"]
    for _, r in df_dc_store.iterrows()
}

In [6]:
# =========================================================
# 4. RESULT STORAGE
# =========================================================
results_plant_dc = []
results_dc_store = []
weekly_costs = []

In [7]:
# =========================================================
# 5. WEEKLY OPTIMIZATION LOOP
# =========================================================
for week in weeks:

    print(f"Solving transport optimization for week {week}")

    week_data = df_demand[df_demand["week"] == week]

    # Create LP model
    model = pulp.LpProblem(f"Transport_Week_{week}", pulp.LpMinimize)

    # -----------------------------
    # Decision Variables
    # -----------------------------
    x = pulp.LpVariable.dicts(
        "x",
        [(i, j, k) for i in plants for j in dcs for k in products],
        lowBound=0
    )

    y = pulp.LpVariable.dicts(
        "y",
        [(j, s, k) for j in dcs for s in stores for k in products],
        lowBound=0
    )

    # -----------------------------
    # Objective Function
    # -----------------------------
    model += (
        pulp.lpSum(
            x[i, j, k] * cost_p2dc.get((i, j, k), 0)
            for i in plants for j in dcs for k in products
        )
        +
        pulp.lpSum(
            y[j, s, k] * cost_dc2s.get((j, s, k), 0)
            for j in dcs for s in stores for k in products
        )
    )

    # -----------------------------
    # Demand Constraints (FORECAST)
    # -----------------------------
    for _, row in week_data.iterrows():
        s = row["store"]
        k = row["product"]
        d = row["forecast_demand"]

        model += pulp.lpSum(y[j, s, k] for j in dcs) == d

    # -----------------------------
    # Flow Conservation at DCs
    # -----------------------------
    for j in dcs:
        for k in products:
            model += (
                pulp.lpSum(x[i, j, k] for i in plants)
                ==
                pulp.lpSum(y[j, s, k] for s in stores)
            )

    # -----------------------------
    # Solve LP
    # -----------------------------
    model.solve(pulp.PULP_CBC_CMD(msg=False))

    # -----------------------------
    # Store Optimal Cost
    # -----------------------------
    optimal_cost = pulp.value(model.objective)

    weekly_costs.append({
        "week": week,
        "optimal_transport_cost": optimal_cost
    })

    print(f"  Optimal cost for week {week}: {optimal_cost:.2f}")

    # -----------------------------
    # Store Flow Results
    # -----------------------------
    for (i, j, k), var in x.items():
        if var.varValue and var.varValue > 0:
            results_plant_dc.append({
                "week": week,
                "from_plant": i,
                "to_dc": j,
                "product": k,
                "quantity": var.varValue
            })

    for (j, s, k), var in y.items():
        if var.varValue and var.varValue > 0:
            results_dc_store.append({
                "week": week,
                "from_dc": j,
                "to_store": s,
                "product": k,
                "quantity": var.varValue
            })

Solving transport optimization for week 2025-09-28
  Optimal cost for week 2025-09-28: 65870.15
Solving transport optimization for week 2025-10-05
  Optimal cost for week 2025-10-05: 65970.35
Solving transport optimization for week 2025-10-12
  Optimal cost for week 2025-10-12: 65902.37
Solving transport optimization for week 2025-10-19
  Optimal cost for week 2025-10-19: 66333.23
Solving transport optimization for week 2025-10-26
  Optimal cost for week 2025-10-26: 66689.61
Solving transport optimization for week 2025-11-02
  Optimal cost for week 2025-11-02: 66656.74
Solving transport optimization for week 2025-11-09
  Optimal cost for week 2025-11-09: 66595.81
Solving transport optimization for week 2025-11-16
  Optimal cost for week 2025-11-16: 67021.18
Solving transport optimization for week 2025-11-23
  Optimal cost for week 2025-11-23: 67362.55
Solving transport optimization for week 2025-11-30
  Optimal cost for week 2025-11-30: 67261.90


In [8]:
# =========================================================
# 6. SAVE RESULTS
# =========================================================
pd.DataFrame(results_plant_dc).to_csv(
    "weekly_transport_paths_plant_dc_forecast.csv", index=False
)

pd.DataFrame(results_dc_store).to_csv(
    "weekly_transport_paths_dc_store_forecast.csv", index=False
)

pd.DataFrame(weekly_costs).to_csv(
    "weekly_optimal_transport_cost.csv", index=False
)

print("\nOptimization completed successfully.")



Optimization completed successfully.


## Plant --> DC

In [9]:
p_dc=pd.DataFrame(results_plant_dc)

In [10]:
p_dc

,week,from_plant,to_dc,product,quantity
0,2025-09-28,plant_1,dc_1,product_1,84.0
1,2025-09-28,plant_1,dc_1,product_15,230.0
2,2025-09-28,plant_1,dc_1,product_5,168.0
3,2025-09-28,plant_1,dc_2,product_1,167.0
4,2025-09-28,plant_1,dc_2,product_10,183.0
...,...,...,...,...,...
625,2025-11-30,plant_3,dc_4,product_14,111.0
626,2025-11-30,plant_3,dc_4,product_6,439.0
627,2025-11-30,plant_3,dc_5,product_16,475.0
628,2025-11-30,plant_3,dc_5,product_18,437.0


## DC --> Store

In [11]:
dc_store=pd.DataFrame(results_dc_store)

In [12]:
dc_store

,week,from_dc,to_store,product,quantity
0,2025-09-28,dc_1,store_1,product_15,165.0
1,2025-09-28,dc_1,store_1,product_20,183.0
2,2025-09-28,dc_1,store_1,product_6,147.0
3,2025-09-28,dc_1,store_1,product_8,199.0
4,2025-09-28,dc_1,store_10,product_12,95.0
...,...,...,...,...,...
1982,2025-11-30,dc_5,store_8,product_15,72.0
1983,2025-11-30,dc_5,store_9,product_13,79.0
1984,2025-11-30,dc_5,store_9,product_2,4.0
1985,2025-11-30,dc_5,store_9,product_20,72.0


## Optimal Transport Cost

In [13]:
cost=pd.DataFrame(weekly_costs)
cost

,week,optimal_transport_cost
0,2025-09-28,65870.15
1,2025-10-05,65970.35
2,2025-10-12,65902.37
3,2025-10-19,66333.23
4,2025-10-26,66689.61
5,2025-11-02,66656.74
6,2025-11-09,66595.81
7,2025-11-16,67021.18
8,2025-11-23,67362.55
9,2025-11-30,67261.90
